In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive/univ/4/nlp/gpt

Mounted at /content/drive
/content/drive/My Drive/univ/4/nlp/gpt


In [ ]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 303.7 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.2 MB/s eta 0:00:00


In [ ]:
from openai import OpenAI
import pandas as pd
import time

# OpenAI API 키 설정
api_key = 'YOUR_API'
client = OpenAI(api_key=api_key)

def get_gpt3_response(prompt):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",  # GPT-3.5 모델 엔진 이름
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=4096,  # 원하는 응답 길이
        n=1,  # 하나의 응답 생성
        stop=None,
        temperature=0.7
    )
    return response.choices[0].message.content

# CSV 파일에서 프롬프트 읽기
csv_file = 'test_data_with_retrieve_and_laws_normalized.csv'
df = pd.read_csv(csv_file)

# 응답 저장을 위한 리스트 초기화
responses = []
start_time = time.time()

# 각 프롬프트에 대해 GPT-3.5 응답 받기
for index, row in df.iterrows():
    prompt = '다음은 내가 처한 상황이야.'+row['자연어 쿼리']+'변호사로서 내 상황에 적용할 수 있는 법령과 그 이유를 설명해줘.'+'다음은 나와 유사한 상황 판례야.'+row['문서 범죄 사실']+'이 판례에서 적용된 법령과 그에 대한 설명이야'+row['문서 법설명']
    response = get_gpt3_response(prompt)
    responses.append(response)
    if (index + 1) % 10 == 0:
        elapsed_time = time.time() - start_time
        print(f'Processed {index + 1} prompts. Elapsed time: {elapsed_time:.2f} seconds.')

# 응답을 새로운 열로 데이터프레임에 추가
df['response'] = responses

# 결과를 새로운 CSV 파일로 저장
output_csv_file = 'baseline4_gpt_result.csv'
df.to_csv(output_csv_file, index=False)

print(f'Responses saved to {output_csv_file}')

Processed 10 prompts. Elapsed time: 75.00 seconds.
Processed 20 prompts. Elapsed time: 158.15 seconds.
Processed 30 prompts. Elapsed time: 240.08 seconds.
Processed 40 prompts. Elapsed time: 325.25 seconds.
Processed 50 prompts. Elapsed time: 397.92 seconds.
Processed 60 prompts. Elapsed time: 478.65 seconds.
Processed 70 prompts. Elapsed time: 563.26 seconds.
Processed 80 prompts. Elapsed time: 652.61 seconds.
Processed 90 prompts. Elapsed time: 743.22 seconds.
Processed 100 prompts. Elapsed time: 835.06 seconds.
Processed 110 prompts. Elapsed time: 936.71 seconds.
Processed 120 prompts. Elapsed time: 1012.43 seconds.
Processed 130 prompts. Elapsed time: 1099.61 seconds.
Processed 140 prompts. Elapsed time: 1186.41 seconds.
Processed 150 prompts. Elapsed time: 1280.80 seconds.
Processed 160 prompts. Elapsed time: 1371.17 seconds.
Processed 170 prompts. Elapsed time: 1461.88 seconds.
Processed 180 prompts. Elapsed time: 1543.96 seconds.
Processed 190 prompts. Elapsed time: 1645.38 seco

In [ ]:
#baseline3-2 gpt f1
import pandas as pd
import numpy as np
from metrics import exact_match_score, accuracy, f1, qa_f1_score

csv_file = 'baseline4_gpt_result.csv'
df = pd.read_csv(csv_file)

# query와 response를 group으로 묶음
df['group'] = df.index // 5

# 각 query마다 F1 점수가 가장 높은 response 선택
best_responses = []

for group_id, group in df.groupby('group'):
    best_f1 = -1
    best_response = None
    ground_truth = group['쿼리 법령의 적용'].iloc[0]  # 같은 query에 대한 ground_truth는 동일하므로 첫번째 값을 사용

    for index, row in group.iterrows():
        response = row['response']
        current_f1 = qa_f1_score(response, ground_truth)

        if current_f1 > best_f1:
            best_f1 = current_f1
            best_response = response

    best_responses.append((group_id, best_response, ground_truth))

# best_responses에서 응답과 정답 추출
preds = [response for _, response, _ in best_responses]
ground_truths = [gt for _, _, gt in best_responses]


# F1 점수 계산
f1_score = f1(preds, [[gt] for gt in ground_truths])
print(f"F1 Score: {f1_score}%")

F1 Score: 14.099046468977042%


In [3]:
!pip install nltk rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=d3abe9044fb51ca1be5ee92956c17cb29307f74750d596c2083b368a107d8f89
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [ ]:
#baseline3-2 gpt
import pandas as pd
import numpy as np
from metrics import exact_match_score, f1, qa_f1_score
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer

csv_file = 'baseline4_gpt_result.csv'
df = pd.read_csv(csv_file)

# query와 response를 group으로 묶음
df['group'] = df.index // 5

# 각 query마다 최고 점수의 response 선택
best_responses = []

# BLEU 및 ROUGE 계산 준비
smoothing = SmoothingFunction().method4
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# 각 메트릭 점수를 저장할 리스트
max_f1_scores = []
max_bleu_scores = []
max_bleu1_scores = []
max_bleu2_scores = []
max_rouge1_scores = []
max_rouge2_scores = []
max_rougeL_scores = []

for group_id, group in df.groupby('group'):
    ground_truth = group['쿼리 법령의 적용'].iloc[0]  # 같은 query에 대한 ground_truth는 동일하므로 첫번째 값을 사용

    max_f1 = -1
    max_bleu = -1
    max_bleu1 = -1
    max_bleu2 = -1
    max_rouge1 = -1
    max_rouge2 = -1
    max_rougeL = -1

    for index, row in group.iterrows():
        response = row['response']

        # F1 점수 계산
        current_f1 = qa_f1_score(response, ground_truth)
        if current_f1 > max_f1:
            max_f1 = current_f1

        # BLEU 점수 계산
        current_bleu = sentence_bleu([ground_truth.split()], response.split(), smoothing_function=smoothing)
        if current_bleu > max_bleu:
            max_bleu = current_bleu

        current_bleu1 = sentence_bleu([ground_truth.split()], response.split(), weights=(1, 0, 0, 0), smoothing_function=smoothing)
        if current_bleu1 > max_bleu1:
            max_bleu1 = current_bleu1

        current_bleu2 = sentence_bleu([ground_truth.split()], response.split(), weights=(0.5, 0.5, 0, 0), smoothing_function=smoothing)
        if current_bleu2 > max_bleu2:
            max_bleu2 = current_bleu2

        # ROUGE 점수 계산
        scores = scorer.score(ground_truth, response)
        if scores['rouge1'].fmeasure > max_rouge1:
            max_rouge1 = scores['rouge1'].fmeasure
        if scores['rouge2'].fmeasure > max_rouge2:
            max_rouge2 = scores['rouge2'].fmeasure
        if scores['rougeL'].fmeasure > max_rougeL:
            max_rougeL = scores['rougeL'].fmeasure

    # 최고 점수를 리스트에 추가
    max_f1_scores.append(max_f1)
    max_bleu_scores.append(max_bleu)
    max_bleu1_scores.append(max_bleu1)
    max_bleu2_scores.append(max_bleu2)
    max_rouge1_scores.append(max_rouge1)
    max_rouge2_scores.append(max_rouge2)
    max_rougeL_scores.append(max_rougeL)

# 각 메트릭의 평균 점수 계산
avg_f1_score = 100 * np.mean(max_f1_scores)
avg_bleu_score = 100 * np.mean(max_bleu_scores)
avg_bleu1_score = 100 * np.mean(max_bleu1_scores)
avg_bleu2_score = 100 * np.mean(max_bleu2_scores)
avg_rouge1_score = 100 * np.mean(max_rouge1_scores)
avg_rouge2_score = 100 * np.mean(max_rouge2_scores)
avg_rougeL_score = 100 * np.mean(max_rougeL_scores)

# 결과 출력
print(f"F1 Score: {avg_f1_score}%")
print(f"BLEU Score: {avg_bleu_score}%")
print(f"BLEU-1 Score: {avg_bleu1_score}%")
print(f"BLEU-2 Score: {avg_bleu2_score}%")
print(f"ROUGE-1 Score: {avg_rouge1_score}%")
print(f"ROUGE-2 Score: {avg_rouge2_score}%")
print(f"ROUGE-L Score: {avg_rougeL_score}%")

F1 Score: 14.099046468977042%
BLEU Score: 1.5362255987973208%
BLEU-1 Score: 7.458959326881392%
BLEU-2 Score: 3.614685043500401%
ROUGE-1 Score: 57.40216732040354%
ROUGE-2 Score: 30.942783696409844%
ROUGE-L Score: 49.89028182177195%


In [4]:
#baseline 3-2 gpt_inter
import pandas as pd
import numpy as np
from metrics import exact_match_score, f1, qa_f1_score
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer

# CSV 파일 읽기
alpha3_csv_file='baseline3_AlphaMist_result.csv'
alpha_csv_file = 'AlphaMist_total_result.csv'
alpha_df = pd.read_csv(alpha_csv_file)
alpha3_df=pd.read_csv(alpha3_csv_file)

csv_file = 'baseline4_gpt_result.csv'
df = pd.read_csv(csv_file)

# 유효한 인덱스 필터링
valid_indices = alpha_df[alpha_df['답변'].notna() & alpha_df['RAG  답변'].notna()&alpha3_df['response'].notna()].index

# group 컬럼 추가
df['group'] = df.index // 5

# BLEU 및 ROUGE 계산 준비
smoothing = SmoothingFunction().method4
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# 각 메트릭 점수를 저장할 리스트
max_f1_scores = []
max_bleu_scores = []
max_bleu1_scores = []
max_bleu2_scores = []
max_rouge1_scores = []
max_rouge2_scores = []
max_rougeL_scores = []

# 각 valid_indices에 대해 계산된 응답의 수를 저장할 리스트
num_responses = []

for idx in valid_indices:
    alpha_query = alpha_df.loc[idx, '자연어 쿼리']
    group = df[df['group'] == idx]

    # 쿼리가 같은지 확인
    df_query = group['자연어 쿼리'].unique()
    if len(df_query) == 1 and alpha_query == df_query[0]:
        ground_truth = group['쿼리 법령의 적용'].iloc[0]  # 같은 query에 대한 ground_truth는 동일하므로 첫번째 값을 사용
        max_f1 = -1
        max_bleu = -1
        max_bleu1 = -1
        max_bleu2 = -1
        max_rouge1 = -1
        max_rouge2 = -1
        max_rougeL = -1

        response_count = 0  # 현재 쿼리에 대해 계산된 응답의 수

        for index, row in group.iterrows():
            response = row['response']
            response_count += 1

            # F1 점수 계산
            current_f1 = qa_f1_score(response, ground_truth)
            if current_f1 > max_f1:
                max_f1 = current_f1

            # BLEU 점수 계산
            current_bleu = sentence_bleu([ground_truth.split()], response.split(), smoothing_function=smoothing)
            if current_bleu > max_bleu:
                max_bleu = current_bleu

            current_bleu1 = sentence_bleu([ground_truth.split()], response.split(), weights=(1, 0, 0, 0), smoothing_function=smoothing)
            if current_bleu1 > max_bleu1:
                max_bleu1 = current_bleu1

            current_bleu2 = sentence_bleu([ground_truth.split()], response.split(), weights=(0.5, 0.5, 0, 0), smoothing_function=smoothing)
            if current_bleu2 > max_bleu2:
                max_bleu2 = current_bleu2

            # ROUGE 점수 계산
            scores = scorer.score(ground_truth, response)
            if scores['rouge1'].fmeasure > max_rouge1:
                max_rouge1 = scores['rouge1'].fmeasure
            if scores['rouge2'].fmeasure > max_rouge2:
                max_rouge2 = scores['rouge2'].fmeasure
            if scores['rougeL'].fmeasure > max_rougeL:
                max_rougeL = scores['rougeL'].fmeasure

        # 최고 점수를 리스트에 추가
        max_f1_scores.append(max_f1)
        max_bleu_scores.append(max_bleu)
        max_bleu1_scores.append(max_bleu1)
        max_bleu2_scores.append(max_bleu2)
        max_rouge1_scores.append(max_rouge1)
        max_rouge2_scores.append(max_rouge2)
        max_rougeL_scores.append(max_rougeL)

        # 응답 수를 리스트에 추가
        num_responses.append(response_count)

# 각 메트릭의 평균 점수 계산
avg_f1_score = 100 * np.mean(max_f1_scores)
avg_bleu_score = 100 * np.mean(max_bleu_scores)
avg_bleu1_score = 100 * np.mean(max_bleu1_scores)
avg_bleu2_score = 100 * np.mean(max_bleu2_scores)
avg_rouge1_score = 100 * np.mean(max_rouge1_scores)
avg_rouge2_score = 100 * np.mean(max_rouge2_scores)
avg_rougeL_score = 100 * np.mean(max_rougeL_scores)

# 결과 출력
print(f"F1 Score: {avg_f1_score}%")
print(f"BLEU Score: {avg_bleu_score}%")
print(f"BLEU-1 Score: {avg_bleu1_score}%")
print(f"BLEU-2 Score: {avg_bleu2_score}%")
print(f"ROUGE-1 Score: {avg_rouge1_score}%")
print(f"ROUGE-2 Score: {avg_rouge2_score}%")
print(f"ROUGE-L Score: {avg_rougeL_score}%")
print(f"Number of queries processed: {len(num_responses)}")
print(f"Number of responses per query: {num_responses}")


F1 Score: 14.823709301441582%
BLEU Score: 1.8888332645585852%
BLEU-1 Score: 8.2332875297838%
BLEU-2 Score: 4.169220685203734%
ROUGE-1 Score: 58.91230938046649%
ROUGE-2 Score: 34.98721678847663%
ROUGE-L Score: 51.61833072441523%
Number of queries processed: 62
Number of responses per query: [5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]


In [5]:
#baseline 3-1 gpt_inter
import pandas as pd
import numpy as np
from metrics import exact_match_score, accuracy, f1, qa_f1_score
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer

# CSV 파일 읽기
alpha3_csv_file='baseline3_AlphaMist_result.csv'
alpha_csv_file = 'AlphaMist_total_result.csv'
alpha_df = pd.read_csv(alpha_csv_file)
alpha3_df=pd.read_csv(alpha3_csv_file)

csv_file = 'baseline4_gpt_result.csv'
df = pd.read_csv(csv_file)

# 유효한 인덱스 필터링
valid_indices = alpha_df[alpha_df['답변'].notna() & alpha_df['RAG  답변'].notna()&alpha3_df['response'].notna()].index

# group 컬럼 추가
df['group'] = df.index // 5

# 각 그룹의 첫 번째 행만 선택
first_rows = df.groupby('group').first().reset_index()

# 유효한 인덱스에 해당하는 행만 필터링
valid_first_rows = first_rows[first_rows['group'].isin(valid_indices)]

# 응답과 정답 추출
preds = valid_first_rows['response'].tolist()
ground_truths = valid_first_rows['쿼리 법령의 적용'].tolist()

# F1 점수 계산
f1_score = f1(preds, [[gt] for gt in ground_truths])
print(f"F1 Score: {f1_score}%")

# BLEU 점수 계산
bleu_scores = [sentence_bleu([gt.split()], pred.split()) for pred, gt in zip(preds, ground_truths)]
bleu_score = 100 * np.mean(bleu_scores)
print(f"BLEU Score: {bleu_score}%")
smoothing = SmoothingFunction().method4

bleu1_scores = [sentence_bleu([gt.split()], pred.split(), weights=(1, 0, 0, 0), smoothing_function=smoothing) for pred, gt in zip(preds, ground_truths)]
bleu2_scores = [sentence_bleu([gt.split()], pred.split(), weights=(0.5, 0.5, 0, 0), smoothing_function=smoothing) for pred, gt in zip(preds, ground_truths)]

bleu1_score = 100 * np.mean(bleu1_scores)
bleu2_score = 100 * np.mean(bleu2_scores)

print(f"BLEU-1 Score: {bleu1_score}%")
print(f"BLEU-2 Score: {bleu2_score}%")
# ROUGE 점수 계산
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
rouge1_scores = []
rouge2_scores = []
rougeL_scores = []

for pred, gt in zip(preds, ground_truths):
    scores = scorer.score(gt, pred)
    rouge1_scores.append(scores['rouge1'].fmeasure)
    rouge2_scores.append(scores['rouge2'].fmeasure)
    rougeL_scores.append(scores['rougeL'].fmeasure)

rouge1_score = 100 * np.mean(rouge1_scores)
rouge2_score = 100 * np.mean(rouge2_scores)
rougeL_score = 100 * np.mean(rougeL_scores)

print(f"ROUGE-1 Score: {rouge1_score}%")
print(f"ROUGE-2 Score: {rouge2_score}%")
print(f"ROUGE-L Score: {rougeL_score}%")


F1 Score: 11.780117491419723%
BLEU Score: 0.8682937008294254%


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

BLEU-1 Score: 6.365362998842759%
BLEU-2 Score: 3.015752221637736%
ROUGE-1 Score: 46.60442745757164%
ROUGE-2 Score: 24.963948326833126%
ROUGE-L Score: 40.14224654122961%


In [ ]:
#baseline3-1 gpt
import pandas as pd
import numpy as np
from metrics import exact_match_score, accuracy, f1, qa_f1_score
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer

# CSV 파일 읽기
csv_file = 'baseline4_gpt_result.csv'
df = pd.read_csv(csv_file)

# query와 response를 group으로 묶음
df['group'] = df.index // 5

first_rows = df.groupby('group').first().reset_index(drop=True)

# first_rows에서 응답과 정답 추출
preds = first_rows['response'].tolist()
ground_truths = first_rows['쿼리 법령의 적용'].tolist()

# F1 점수 계산
f1_score = f1(preds, [[gt] for gt in ground_truths])
print(f"F1 Score: {f1_score}%")

bleu_scores = [sentence_bleu([gt.split()], pred.split()) for pred, gt in zip(preds, ground_truths)]
bleu_score = 100 * np.mean(bleu_scores)
print(f"BLEU Score: {bleu_score}%")
smoothing = SmoothingFunction().method4

bleu1_scores = [sentence_bleu([gt.split()], pred.split(), weights=(1, 0, 0, 0), smoothing_function=smoothing) for pred, gt in zip(preds, ground_truths)]
bleu2_scores = [sentence_bleu([gt.split()], pred.split(), weights=(0.5, 0.5, 0, 0), smoothing_function=smoothing) for pred, gt in zip(preds, ground_truths)]

bleu1_score = 100 * np.mean(bleu1_scores)
bleu2_score = 100 * np.mean(bleu2_scores)

print(f"BLEU-1 Score: {bleu1_score}%")
print(f"BLEU-2 Score: {bleu2_score}%")

# ROUGE 점수 계산
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
rouge1_scores = []
rouge2_scores = []
rougeL_scores = []

for pred, gt in zip(preds, ground_truths):
    scores = scorer.score(gt, pred)
    rouge1_scores.append(scores['rouge1'].fmeasure)
    rouge2_scores.append(scores['rouge2'].fmeasure)
    rougeL_scores.append(scores['rougeL'].fmeasure)

rouge1_score = 100 * np.mean(rouge1_scores)
rouge2_score = 100 * np.mean(rouge2_scores)
rougeL_score = 100 * np.mean(rougeL_scores)

print(f"ROUGE-1 Score: {rouge1_score}%")
print(f"ROUGE-2 Score: {rouge2_score}%")
print(f"ROUGE-L Score: {rougeL_score}%")

F1 Score: 10.49078617479166%
BLEU Score: 0.502025380754265%


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

BLEU-1 Score: 5.346126828319042%
BLEU-2 Score: 2.406951055869683%
ROUGE-1 Score: 43.49382991505471%
ROUGE-2 Score: 20.37837296374267%
ROUGE-L Score: 37.37995275471785%
